In [ ]:
# sekcja serwisowa
import spacy
nlp = spacy.load("pl_core_news_lg")
import pandas as pd
df = pd.DataFrame
pd.set_option('display.max_rows', None) # nie skracaj tabeli df
pd.set_option('max_colwidth', 500) # maksymalna dopuszczalna szerokość kolumny df 

In [ ]:
# PODAJ STOPWORD
stopword = "stop"
# PODAJ TYTUŁ
title = "System zarządzania sklepem"
# PODAJ ŚCIEŻKĘ DO PLIKU TEKSTOWEGO - ZAPISU ROZMOWY
source = "klient1.txt"

In [ ]:
# zwrotka dla użytkownika
f = open(source, "r", encoding="utf-8")
tekst = f.read()

if stopword in tekst:
    print("Wybierz inny stopword!")
else:
    doc = nlp(tekst)
    print("OK!")

In [ ]:
# wyświetla token oraz przypisaną do niego część mowy, dep i head
colttext, coltpos, coltdep, colhdtx = "", "", "", ""
tokcol1, tokcol2, tokcol3, tokcol4 = "", "", "", ""

for token in doc:
    tokcol1 += token.text + stopword
    tokcol2 += token.pos_ + stopword
    tokcol3 += token.dep_ + stopword
    tokcol4 += token.head.text + stopword
colttext = tokcol1.split(stopword)
coltpos = tokcol2.split(stopword)
coltdep = tokcol3.split(stopword)
colhdtx = tokcol4.split(stopword)
tok = {'token' : colttext, 'część mowy' : coltpos, 'dep' : coltdep, 'head' : colhdtx}

df(tok)

In [ ]:
# zasady służące do wyłapywania w tekście akcji i aktorów - serce skryptu
podmioty, orzeczenia, zdania = [], [], []

for token in doc:
    # wyłapuje podmioty które są rzeczownikiem lub nazwą własną
    if token.dep_ == "nsubj" and (token.pos_ == "NOUN" or token.pos_ == "PROPN"):
        podmioty.append(token.text.lower())
        orzeczenia.append(token.head.text.lower())
        zdania.append(token.sent)
        
    # wyłapuje czasowniki
    elif token.pos_ == "VERB":
        orzeczenia.append(token.text.lower())
        podmioty.append("null")
        zdania.append(token.sent)
    
    # wyłapuje czasowniki posiłkowe CHYBA
    elif token.pos_ == "AUX":
        orzeczenia.append(token.text.lower() + " " + token.head.text.lower())
        podmioty.append("null")
        zdania.append(token.sent)
        
    # wyłapuje czasowniki posiłkowe
    elif token.pos_ == "PRON" and token.dep_ == "expl:pv":
        orzeczenia.append(token.head.text.lower() + " " + token.text.lower())
        podmioty.append("null")
        zdania.append(token.sent)

In [ ]:
# eksportuje do .xlsx wynik
oip = {}
oip = {"Aktorzy" : podmioty, "Akcje" : orzeczenia, "Zdanie" : zdania}
df(oip).to_excel("./data.xlsx")

In [ ]:
# tutaj konieczna jest interwencja użytkownika

In [ ]:
# importuje z .xlsx, usuwa zduplikowane wiersze
excel = pd.read_excel("./data.xlsx")
df2 = df.drop_duplicates(excel)
df2

In [ ]:
# usuwa kopie gdzie brakuje czasowników posiłkowych
aktorzy = list(df2['Aktorzy'])
akcje = list(df2['Akcje'])
aktorzy.append(stopword)
aktorzy.append(stopword)
akcje.append(stopword)
akcje.append(stopword)

i=0
for i in range (0,len(df2['Akcje'])):
    if aktorzy[i] == aktorzy[i+1]:
        if akcje[i] == (akcje[i+1] + " się"):
            aktorzy.remove(aktorzy[i+1])
            akcje.remove(aktorzy[i+1])
        elif (akcje[i] + " się") == akcje[i+1]:
            aktorzy.remove(aktorzy[i])
            akcje.remove(akcje[i])
        i += 1
        
for i in range(0,len(aktorzy)):
    if stopword in aktorzy:
        aktorzy.remove(stopword)
    elif stopword in akcje:
        akcje.remove(stopword)


In [ ]:
# tabela aktorów, przypisanych do nich akcji i zdań w których występują
final = {}
final = {"Aktorzy" : aktorzy, "Akcje" : akcje, "Zdanie" : zdania}
df(final)

In [ ]:
# generuje wykres z aktorami i przypisanymi akcjami
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

dlugosc = len(aktorzy)
if dlugosc > 8:
    dlugosc = 8
i=0

ax.text(0.25, 0.95, title, color = 'black')

for i in range(0, dlugosc):
    aktor = aktorzy[i]
    akcja = akcje[i]
    ax.text(0.3, 0.8 - 0.1*i, aktor + ' ---> ' + akcja, color='black')
    i += 1

plt.show()

In [ ]:
# wyjaśnia skróty
spacy.explain("PRON")

In [ ]:
# generuje wykres
from spacy import displacy
displacy.render(doc, style="dep", jupyter=True) # ent/dep

In [ ]:
# nazwy własne np miejsca, organizacje
colenttext, colentlabel, entcol1, entcol2 = "", "", "", ""

for ent in doc.ents:
    entcol1 += ent.text + stopword
    entcol2 += ent.label_ + stopword
colenttext = entcol1.split(stopword)
colentlabel = entcol2.split(stopword)
ent = {'tekst' : colenttext, 'etykieta' : colentlabel}

df(ent)

In [ ]:
# morfologia
colmortext, colmormor, morcol1, morcol2 = "", "", "", ""

for token in doc:
    morcol1 += token.text + stopword
    morcol2 += str(token.morph) + stopword
colmortext = morcol1.split(stopword)
colmormor = morcol2.split(stopword)
morph = {'tekst' : colmortext, 'morfologia' : colmormor}

df(morph)

In [ ]:
# lemmatizacja
collemtext, collemlem, lemcol1, lemcol2 = "", "", "", ""

for token in doc:
    lemcol1 += token.text + stopword
    lemcol2 += str(token.lemma_) + stopword
collemtext = lemcol1.split(stopword)
collemlem = lemcol2.split(stopword)
lemma = {'tekst' : collemtext, 'lemmatyzacja' : collemlem}

df(lemma)

In [ ]:
# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)

In [ ]:
# for token in doc:
#     print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])